# FEM Notebook 2: Introduction to the Finite Element Method

## Introduction
Welcome to the second notebook in my series "Understanding and Using the Finite Element Method". We now have a strong enough mathematical foundation to begin introducing new mathematics, which will help us define the algorithm that is FEM. We will start in 1-dimensional problems, with domains that lie strictly on the x-axis, and work our way up to 2.

### Learning Objectives

- To understand the space of piecewise continuous linear polynomials $V_h$ and its basis.
- To be able to construct and solve a finite element problem in 1D.

## 3. Finite Element Method in 1D

### 3.1 What is the goal of FEM?

Finite Element Analysis is a tool for solving partial differential equations. For example consider the equation 

$$ -\frac{\mathrm{d}^2 u}{\mathrm{d} x^2} = f(x) \,, \quad u(0) = 0 \,,\, u(L) = 0 \,, \quad 0 \leq x \leq L \tag{1}$$ 

where $u$ and $f$ are single variable functions of $x$. Since, for now, we are only concerned with 1-dimensional functions, partial differential equations are the same as differential equations, because there is only one variable to differentiate with respect to. For most $f$, we can easily solve this by integrating twice, but it will serve as an example for this chapter. We can check the difference between our answer and analytical solution, and calculate the error between the two. More importantly however, this approach will work even for complicated functions that we cannot integrate.

The goal of FEM is to produce a suitable approximation of $u$ that solves (1). It is done by splitting the domain up into a set of small regions called a mesh. Then, we assume our approximation of $u$, denoted $u_h$ is a combination of straight lines, one from each of the individual regions on our mesh. We can also rewrite (1) in a new way, called the variational form. Finally, we insert our $u_h$ in for $u$, which provides us with a system of linear equations we can solve, to find the values of our approximated function. This is the general outline of how FEM works in 1D, and the higher dimensional versions work very similarly too!

We need to start by defining the space our approximated solution comes from. This space is called the space of **Piecewise Continuous Linear Polynomials**.

### 3.2 The Space of Piecewise Continous Linear Polynomials

All of our PDEs will be solved over an interval for example $I = [0, L]$. The square brackets indicate that the end points are included. Our approach involves splitting the interval into a **mesh**, which is a collection of $n$ equally-sized sub-intervals, at the points $\{x_i\}_{i=0}^{n-1}$. Fig 5 shows an example of a simple 1-dimensional mesh on a numberline. Each sub-interval is of size $1$, leading to $x_i=i$ for $i=0,\cdots,L$. 

<img src="1d_mesh.PNG" style="display:block;margin-left:auto;margin-right:auto;width:50%">
<figcaption style="font-style:italic;text-align:center;">Fig.5 - Interval on the numberline</figcaption>

Let's focus on a single sub-interval $J = [x_0, x_1]$. Since our solution should be a collection of straight lines, this sub-interval should contain a linear polynomial. We encountered the space of polynomials of at most degree $n$ in section 1.1, so for linear polynomials we have $P_1(J)$. Functions from $P_1(J)$ take the form $v(x) = c_0 + c_1x$. The most obvious choise of basis would therefore be the standard basis $\{ 1, x \}$, since any $v\in P_1(I_i)$ could be specified by only $c_0$ and $c_1$. 

This is not the only basis we can use, and for our purposes it makes more sense to go with a different one. You should be aware that any straight line can be defined by two points it passes through. So if the endpoints of $J$ are $x_0$ and $x_1$ (called the **nodes** of $J$) $v$ can be specified by a line that passes through $v(x_0)=\alpha_0$ and $v(x_1)=\alpha_1$, if these values are known. We can construct a basis where $a_0$ and $a_1$ are the coefficients that can be used to specify any $v\in P_1(J)$. This basis is called the **nodal basis** of $P_1$ and is defined as

$$
\lambda_j(x_i) =
  \begin{cases}
    1 & \text{if $i=j$}\\
    0 & \text{if $i \neq j$}
  \end{cases}
  \quad i,j = 0,1
$$

Then we can write $v(x)=\alpha_0\lambda_0(x) + \alpha_1\lambda_1(x)$. So if $x=x_0$, $v(x)=\alpha_0$ and if $x=x_1$, $v(x)=\alpha_1$ as required. If $x$ is between $x_0$ and $x_1$, the function should linearly interpolate between these values. The explicit form of this basis is 

$$
\lambda_0(x) = \frac{x_1 - x}{x_1 - x_0}, \quad \lambda_1(x) = \frac{x - x_0}{x_1 - x_0}
$$

Now we have this basis for $P_1(J)$. This can now be extended to include the whole mesh, which will be a collection of linear functions. These functions are called 'piecewise' linear, because when considered in individual pieces, a function is linear, but overall the function is not. One condition we must impose is that these piecewise linear functions are continuous. This is because the solutions to all PDEs we will be solving will also be continuous. Below are two examples of piecewise linear functions. One is continous and the other is not.

<table><tr>
<td> <img src="Discontinuous_piecewise_linear.png" alt="Drawing"/> 
    <figcaption style="font-style:italic;text-align:center;">Fig.6 - Discontinuous Piecewise Linear Function</figcaption> </td>
<td> <img src="Continuous_piecewise_linear.png" alt="Drawing"/> 
    <figcaption style="font-style:italic;text-align:center;">Fig.7 - Continuous Piecewise Linear Function</figcaption> </td>
</tr></table>

We are now in a position to properly define the vector space over which our solution will be defined. This is 

$$ V_h = \Big\{ v \, : \, v \in C^0(I) \, , \,\, v|_{I_i}\in P_1(I_i)\Big\}.$$

Here, $C^0(I)$ is the set of continuous functions defined on the interval $I$. So $V_h$ includes functions that are continuous on $I$ and linear on each subinterval $I_i$. Once again we define $I = [0, L]$ with $\{x_i\}_{i=0}^{n-1}$ being the nodes of our mesh. The basis for this vector space isn't too dissimilar from the nodal basis of $P_1$. In this case, it is defined as

$$
\phi_j(x_i) =
  \begin{cases}
    1 & \text{if $i=j$}\\
    0 & \text{if $i \neq j$}
  \end{cases}
  \quad i,j = 0,1,\cdots,n
$$

Or, in it's explicit form

$$
\phi_i(x) =
  \begin{cases}
    (x-x_{i-1})\,/\,h_i & \text{if $x \in I_i$}\\
    (x_{i+1}-x)\,/\,h_{i+1} & \text{if $x \in I_{i+1}$}\\
    0 & \text{otherwise}
  \end{cases}
$$

This nodal basis has been designed specifically so that any function $v$ can be written as a linear combination of $\phi$'s, where the coefficient of $\phi_i(x)$ is $v(x_i)$. Specifically, $v(x) = \sum_{i=0}^{n-1} \phi_i(x)v(x_i)$. These functions $\phi_i(x)$ are often called **hat functions** due to their shape (seen below), while the hat functions closest to the edge of the interval are called **half-hats**.

<table><tr>
<td> <img src="Half_hat.png" alt="Drawing"/> 
    <figcaption style="font-style:italic;text-align:center;">Fig.8 - Half-Hat</figcaption> </td>
<td> <img src="Hat.png" alt="Drawing"/> 
    <figcaption style="font-style:italic;text-align:center;">Fig.9 - Hat</figcaption> </td>
</tr></table>

### 3.3 Variational Formulation

Let's return to our initial example.

$$ -\frac{\mathrm{d}^2 u}{\mathrm{d} x^2} = f(x) \,, \quad u(0) = 0 \,,\, u(L) = 0 \,, \quad 0 \leq x \leq L \tag{1}$$

One of the key tricks to Finite Element Analysis is through converting a partial differential equation into a **variational** or **weak** form. This is done by multiplying both sides of the equation by a **test function** $v$. It should be mentioned that $u$, the function we are solving for, is often called the **trial function**. We will assume the test function vanishes at the end-points of our interval. We do this multiplication, then integrate by parts to yield the weak formulation of (1).

$$ 
\begin{align*}
-vu'' &= vf \\
\int_0^L vf \, \mathrm{d}x &= -\int_0^L vu'' \mathrm{d}x \\
&= -\Big[vu'\Big]_0^L + \int_0^Lv'u'\,\mathrm{d}x \\
&= -v(0)u'(0) + v(L)u'(L) + \int_0^Lv'u'\,\mathrm{d}x \\
&= \int_0^Lv'u'\,\mathrm{d}x, \quad \forall v \in V
\end{align*}
$$

We should add a quick side note to talk about the function spaces involved here. In order for the above to be true, we must have that $v$ and $v'$ are **square integrable** on $I$. For some function $f$ to be square integrable, the integral $\int_I |f(x)|^2 \, \mathrm{d}x$ must be finite. The space of square integrable functions defined on some interval $I$ is called $L^2(I)$, which is an **inner product space** defined with the inner product $\langle f, g \rangle = \int_I \bar{f}g \, \mathrm{d}x$. Since $\bar{f}f=|f|^2$, an equivalent definition of square integrability is that $\langle f, f \rangle_{L^2(I)} < \infty$. It then follows that $L^2(I)$ is a **Hilbert Space**. So, in order for the above to be true, we must have $v, v' \in L^2(I)$, but this is precisely the definition of the **Sobloev space** $H^1(I)$ equipped with the $L^2$ inner product. A Sobloev space is just a Hilbert space where derivatives must be square integrable as well. Therefore, we must have $v \in H^1(I)$. 

This is a glimpse into the world of functional analysis, and while it is certainly interesting, you might be wondering how this pertains to solving PDEs? Well, the useful thing about $H^1$ is that it permits functions that have discontinuous derivatives. This is extremely useful, because our piecewise continuous linear polynomials have discontinuous derivatives. You can take a look again at Fig.7 to convince yourself this is true. This means we can transform our weak formulation, which at the minute is a continuous problem, into one that allows for piecewise polynomials to approximate it. Hence, we define the space

$V$